In [22]:
import numpy as np
import pandas as pd
import json
from model import StreetConfig, Street
import agents

with open('conf.json') as f:
    conf = json.load(f)
    conf = StreetConfig(**conf)

## Street Config Object?

- `name` values must be unique for each dictionary in `parks` list.
- `p_exist` values must sum up to 1.0.

In [2]:
conf.__class__

model.StreetConfig

In [3]:
conf

{'parks': [{'cost': 3.0, 'p_occupied': 0.6, 'p_exist': 0.4, 'name': 'green'},
  {'cost': 5.0, 'p_occupied': 0.4, 'p_exist': 0.6, 'name': 'red'}],
 'cost_garage': 10.0,
 'cost_walk_unit': 0.1}

In [4]:
conf.cost_garage

10.0

In [5]:
conf.cost_walk_unit

0.1

## Parking Spaces?

In [6]:
conf.parks

[ParkConf(cost=3.0, p_occupied=0.6, p_exist=0.4, name='green'),
 ParkConf(cost=5.0, p_occupied=0.4, p_exist=0.6, name='red')]

In [7]:
conf.parks.__class__

model.ParkList

In [8]:
conf.parks.to_frame()

,cost,p_occupied,p_exist,name
0,3.0,0.6,0.4,green
1,5.0,0.4,0.6,red


### Marginal Probabilties empty spaces

In [9]:
conf.parks.empty_marginal()

{'green': 0.30769230769230776, 'red': 0.6923076923076923}

### Marginal probabilty of being occupied

In [10]:
p = conf.parks.p_occupied()
p, 1- p

(0.48, 0.52)

### Getting attributes as a dict

Use `ParkingOnStreet` attributes on `ParkList` to get a dict of contained parking spaces

In [11]:
conf.parks.cost

{'green': 3.0, 'red': 5.0}

In [12]:
conf.parks.name

{'green': 'green', 'red': 'red'}

### Park Object

In [13]:
conf.parks[0].__class__

model.ParkConf

In [14]:
conf.parks[0]

ParkConf(cost=3.0, p_occupied=0.6, p_exist=0.4, name='green')

In [15]:
conf.parks[0].cost

3.0

### Street Object

In [16]:
conf.parks

[ParkConf(cost=3.0, p_occupied=0.6, p_exist=0.4, name='green'),
 ParkConf(cost=5.0, p_occupied=0.4, p_exist=0.6, name='red')]

In [17]:
[park for park in conf.parks][0]

ParkConf(cost=3.0, p_occupied=0.6, p_exist=0.4, name='green')

In [35]:
N = 70
s = Street(N, conf.parks, conf.cost_garage, conf.cost_walk_unit)

Also possible to pass conf as a mapping

In [36]:
s = Street(N, **conf)

### Agents

All agents share attributes `state`, `spaces_left`, `done` and `conf`.  
`observe` method is used to parse the attributes above and `decide` for deciding on the action.

In [37]:
agent = agents.ThresholdAgent(conf, threshold=8)

In [38]:
s.reset()

(0, 0.0, False)

### One step of state -> action -> reward -> state -> action ...

In [45]:
agent.observe(*s.reset())
action = agent.decide()
state, cost, done, spaces_left = s.step(action)
agent.observe(state, spaces_left, done)
agent.decide()

0

### Going through the street with a loop

In [257]:
done = False
state, cost, done = s.reset()
agent = agents.ThresholdAgent(conf, threshold=4)
agent.observe(state, s.spaces_left, done)
while not done:
    action = agent.decide()
    state, cost, done, spaces_left = s.step(action)
    agent.observe(state, spaces_left, done)
print('Final cost:', cost)
print('Best possible:', s.best_possible_cost)
print("Missed opportunity:", cost - s.best_possible_cost)

Final cost: 5.1
Best possible: 3.2
Missed opportunity: 1.8999999999999995


### Going through the street with a loop

In [305]:
done = False
state, cost, done = s.reset()
agent = agents.ExpectedAgent(conf)
agent.observe(state, s.spaces_left, done)
while not done:
    action = agent.decide()
    state, cost, done, spaces_left = s.step(action)
    agent.observe(state, spaces_left, done)
print('Final cost:', cost)
print('Best possible:', s.best_possible_cost)
print("Missed opportunity:", cost - s.best_possible_cost)

Final cost: 4.2
Best possible: 3.4
Missed opportunity: 0.8000000000000003
